In [1]:
import geopandas as gpd
import pandas as pd
from matplotlib import pyplot as plt

In [2]:
# Load data into panda DataFrame
df = pd.read_csv(r'..\BADENOVAnetze Daten\Strom-Einspeiser-Export 1.csv', encoding='ansi', sep=';', on_bad_lines='skip')

FileNotFoundError: [Errno 2] No such file or directory: '..\\BADENOVAnetze Daten\\Strom-Einspeiser-Export 1.csv'

In [ ]:
# Parse date
df['Einbaudatum'] = pd.to_datetime(df['Einbaudatum'], errors='coerce', dayfirst=True)

# Only keep Not-Null entries
df = df[df['Einbaudatum'].notnull()]  

# extract year and month
df['Jahr'] = df['Einbaudatum'].dt.year
df['Monat'] = df['Einbaudatum'].dt.month

In [ ]:
# Dictionary with building counts, partially approximated with population divided by average amount of people per house. Sources below.
anzahl_gebaeude = {
    "Freiburg": 32249,
    "Gundelfingen": 2686,
    "Kirchzarten": 2260,
    "Bad Krozingen": 4829,
    "Breisach": 3671,
    "Buggingen": 985,
    "Ebringen": 646,
    "Gottenheim": 687,
    "Heitersheim": 1430,
    "Ihringen": 1404,
    "Lauchringen": 1790,
    "Merdingen": 590,
    "Müllheim": 4437,
    "Neuenburg": 2839,
    "St. Peter": 608,
    "Staufen": 1884,
    "Stegen": 997,
    "Titisee-Neustadt": 2659,
    "Umkirch": 1108,
    "Weisweil": 466,
    "Wutöschingen": 1330
}

# Convert to pandas Dataframe
gebaeude_df = pd.DataFrame(list(anzahl_gebaeude.items()), columns=["Gemeinde", "Anzahl_Haeuser"])

# Merge with dataframe
df = df.merge(gebaeude_df, on="Gemeinde", how="left")


In [ ]:
# Grouping by community, year and month
gruppen = df.groupby(['Gemeinde', 'Jahr', 'Monat']).size().reset_index(name='Anzahl')

# Create date column for visualization
gruppen['Datum'] = pd.to_datetime(
    gruppen['Jahr'].astype(str) + '-' +
    gruppen['Monat'].astype(str).str.zfill(2) + '-01'
)

# Filtering data starting with 2001
gruppen = gruppen[gruppen['Datum'].dt.year >= 2001]

# Grouping by communitys
gemeinden = df['Gemeinde'].value_counts().index
gruppen = gruppen[gruppen['Gemeinde'].isin(gemeinden)]

# Total house count per community
gruppen = gruppen.merge(gebaeude_df, on="Gemeinde", how="left")

# Cumulative count
gruppen['Kumuliert'] = gruppen.groupby('Gemeinde')['Anzahl'].cumsum()

# Cumulative relative proportion
gruppen['Kumuliert_Prozent'] = (gruppen['Kumuliert'] / gruppen['Anzahl_Haeuser']) * 100

# Plot
plt.figure(figsize=(14, 7))

for gemeinde, gruppe in gruppen.groupby('Gemeinde'):
    plt.plot(gruppe['Datum'], gruppe['Kumuliert_Prozent'], label=gemeinde)

plt.title('Kumulierte Einbauten in % pro Monat nach Gemeinde (ab 2001)')
plt.xlabel('Monat')
plt.ylabel('Kumulierte Einbauten in % der Gebäude')
plt.legend(loc='upper left', fontsize='small', ncol=2)
plt.grid(True)
plt.xticks(rotation=45)
plt.xlim(pd.to_datetime('2001-01-01'), pd.to_datetime('2024-12-31'))
plt.tight_layout()
plt.show()


## 2000–2003: 100.000-Dächer-Programm

Dieses Programm bot zinsgünstige Kredite für die Installation von Photovoltaikanlagen und war ein bedeutender Anreiz für Privatpersonen und kleine Unternehmen. Es endete 2003, nachdem das Ziel von 300 MW installierter Leistung erreicht wurde. 
Wikipedia

## Ab 2000: Erneuerbare-Energien-Gesetz (EEG)

Das EEG garantierte feste Einspeisevergütungen über 20 Jahre für eingespeisten Solarstrom, was Investitionssicherheit schuf. Die Vergütungssätze wurden jedoch regelmäßig angepasst, um Marktveränderungen zu berücksichtigen. 
FNP
+4
Grünes Haus
+4
Wikipedia
+4
Grünes Haus
+2
FNP
+2
Wikipedia
+2

## 2012: Einführung des 52-GW-Deckels

Eine Obergrenze von 52 GW förderbarer Solarleistung wurde eingeführt, was zu einem Rückgang des Zubaus führte. Diese Begrenzung wurde 2020 aufgehoben, um den weiteren Ausbau zu ermöglichen. 
Wikipedia

## 2022: Osterpaket

Mit dem Osterpaket wurden Änderungen im EEG beschlossen, darunter die Abschaffung der 70%-Regelung für Neuanlagen bis 25 kW und eine Anpassung der Degressionsmechanismen für die Einspeisevergütung. 
FNP
+3
BSH Energie
+3
Wikipedia
+3

## 2023: Nullsteuersatz für Photovoltaik

Seit dem 1. Januar 2023 gilt ein Nullsteuersatz für Photovoltaikanlagen bis 30 kWp, wodurch keine Umsatzsteuer auf Kauf und Installation anfällt. Dies senkte die Anschaffungskosten erheblich. 
G

## Sources
https://www.freiburg.de/pb/1649489.html  
https://www.statistischebibliothek.de/mir/servlets/MCRFileNodeServlet/BWHeft_derivate_00009738/083155005047_Gundelfingen_GWZ.pdf  
https://d-nb.info/1229363238/34  
https://www.statistischebibliothek.de/mir/servlets/MCRFileNodeServlet/BWHeft_derivate_00009718/083155002015_Breisach_am_Rhein_Stadt_GWZ.pdf  
https://wahlatlas.net/experimente/zensus2022/gemeinden/083155009043.html  
https://www.st-peter-schwarzwald.de/fileadmin/dokumente/Zensusergebnis_Gebaeude.pdf  
https://www.citypopulation.de/de/germany/badenwurttemberg/breisgau_hochschwarzwald/08315047__gundelfingen/  
https://de.wikipedia.org/wiki/Kirchzarten  
https://de.wikipedia.org/wiki/Bad_Krozingen  
https://stadt.breisach.de/de/stadtinfo/zahlen_daten_fakten/einwohnerzahlen  
https://buggingen.de/de/home  
https://en.wikipedia.org/wiki/Ebringen  
https://www.gottenheim.de/Tourismus/Overview/  
https://de.wikipedia.org/wiki/Heitersheim  
https://www.ihringen.de/unser-ihringen/daten%2Bund%2Bfakten  
https://de.wikipedia.org/wiki/Lauchringen  
https://de.wikipedia.org/wiki/Merdingen  
https://www.muellheim.de/stadtentwicklung-wirtschaftsfoerderung/stadtinfos/einwohnerzahlen/  
https://www.neuenburg.de/Startseite/wirtschaft%2Bbauen/Neuenburg%2Bin%2BZahlen.html  
https://www.st-peter-schwarzwald.de/buergerservice/buergerinformationen/finanzen.html  
https://de.wikipedia.org/wiki/Staufen_im_Breisgau  
https://de.wikipedia.org/wiki/Stegen_(Hochschwarzwald)  
https://de.wikipedia.org/wiki/Titisee-Neustadt  
https://de.wikipedia.org/wiki/Umkirch  
https://de.wikipedia.org/wiki/Weisweil  
https://de.wikipedia.org/wiki/Wutöschingen  
https://www.baden-wuerttemberg.de/de/unser-land/land-und-leute/bevoelkerung#:~:text=In%20Baden-Württemberg%20leben%20rund,1%20Millionen%20Menschen%20im%20Südwesten  
https://www.statistik-bw.de/Wohnen/GebaeudeWohnungen/BW-BT_einfamilienhaeuser.jsp#:~:text=Ende%202022%20gab%20es%20in,Zweifamilienhäuser%20und%20knapp%2018%20%%20Mehrfamilienhäuser